# Cifar10 model testing
In this notebook we will test if working with grey-scaled images gives better results than working with RGB images. For this, we will try several techniques:
- Flatten the three channels **(Flatten)**
- Maximum of the three channels **(MaxChannels)**
- Use the image with three channels **(Embedding)**
- Convolutional layer **(Convolutional)**
- Black and white function **(BlackWhite)**

# Make the necessary imports

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import math
import shutil
import os
import gc

# Import and preprocess Cifar10 dataset

In [ ]:
(Xtrain_orig, ytrain_orig), (Xtest_orig, ytest_orig) = cifar10.load_data()

Xtrain_norm = Xtrain_orig.astype('float32') / 255.
Xtest_norm = Xtest_orig.astype('float32') / 255.

ytrain_norm = ytrain_orig
ytest_norm = ytest_orig

print ("Xtrain shape: " + str(Xtrain_norm.shape))
print ("ytrain shape: " + str(ytrain_norm.shape))

print ("Xtest shape: " + str(Xtest_norm.shape))
print ("ytest shape: " + str(ytest_norm.shape))

del Xtrain_orig
del ytrain_orig
del Xtest_orig
del ytest_orig

# Define some variables

In [ ]:
latent_dim = 32
img_size = 32

# Show an image of every class

In [ ]:
f, axarr = plt.subplots(1, 10, figsize=(30, 10))
for x in range(10):
    idx = np.where(ytrain_norm==x)[0][0]
    axarr[x].imshow(Xtrain_norm[idx])


# Support functions

In [ ]:
def create_set(X, y, strip_size=4, set_size=60000):
    
    #Create a list of lists where every sublist contains the indexes of the images belonging to a class
    list_indices_by_number = [np.where(y == i)[0] for i in range(10)]
    
    #Create the strips of images
    X_groups = []
    number_groups = []
    y_label = []
    
    for i in range(set_size): #Create as many images as strip_size
        group_i = []
        numbers_i = []
        while len(group_i) < strip_size: #While the strip is shorter that the size wanted
            #Choose a random index
            image_idx = random.randint(0, len(X)-1)
            numbers_i.append(y[image_idx][0])
            group_i.append(image_idx)
        #When the strip is full, add the target image. Use random to obtain a balanced set.
        repeated = np.random.choice([0, 1], p=[0.50, 0.50])
        if repeated:
            #Look for a number whose class is already contained in the strip.
            random_idx = random.randint(0, len(numbers_i)-1)
            number = numbers_i[random_idx]
            numbers_i.append(number)
            #Choose a random image representing the chosen class
            image_idx = random.randint(0, len(list_indices_by_number[number])-1)
            group_i.append(list_indices_by_number[number][image_idx])
            y_label.append(1)
        else:
            #Add a number that is not aready in the strip
            possible_numbers = [x for x in range(10) if x not in numbers_i]
            random_number = random.choice(possible_numbers)
            numbers_i.append(random_number)
            #Choose a random image representing the chosen class
            image_idx = random.randint(0, len(list_indices_by_number[random_number])-1)
            group_i.append(list_indices_by_number[random_number][image_idx])
            y_label.append(0)
        X_groups.append(group_i)
        number_groups.append(numbers_i)
    
    #We now want our examples to have the following shape: (N, strip_size+1, X_train[1], X_train[2], 3)
    #And create the expected labels
    N = len(X_groups)
    img_size1 = X.shape[1]
    img_size2 = X.shape[2]
    X_processed= np.zeros([N, strip_size+1, img_size1, img_size2, 1])
    y_processed = np.zeros([N])
    for i in range(N):
        numbers_i = list(dict.fromkeys(number_groups[i]))
        for j in range(strip_size):
            X_processed[i, j:j+1, :, :, :] = tf.expand_dims(X[X_groups[i][j]], axis=-1)
        X_processed[i, strip_size, :, :, :] = tf.expand_dims(X[X_groups[i][strip_size]], axis=-1)
        y_processed[i] = y_label[i]
        
    return X_processed, y_processed

In [ ]:
def create_set_RGB(X, y, strip_size=4, set_size=60000):
    
    #Create a list of lists where every sublist contains the indexes of the images belonging to a class
    list_indices_by_number = [np.where(y == i)[0] for i in range(10)]
    
    #Create the strips of images
    X_groups = []
    number_groups = []
    y_label = []
    
    for i in range(set_size): #Create as many images as strip_size
        group_i = []
        numbers_i = []
        while len(group_i) < strip_size: #While the strip is shorter that the size wanted
            #Choose a random index
            image_idx = random.randint(0, len(X)-1)
            numbers_i.append(y[image_idx][0])
            group_i.append(image_idx)
        #When the strip is full, add the target image. Use random to obtain a balanced set.
        repeated = np.random.choice([0, 1], p=[0.50, 0.50])
        if repeated:
            #Look for a number whose class is already contained in the strip.
            random_idx = random.randint(0, len(numbers_i)-1)
            number = numbers_i[random_idx]
            numbers_i.append(number)
            #Choose a random image representing the chosen class
            image_idx = random.randint(0, len(list_indices_by_number[number])-1)
            group_i.append(list_indices_by_number[number][image_idx])
            y_label.append(1)
        else:
            #Add a number that is not aready in the strip
            possible_numbers = [x for x in range(10) if x not in numbers_i]
            random_number = random.choice(possible_numbers)
            numbers_i.append(random_number)
            #Choose a random image representing the chosen class
            image_idx = random.randint(0, len(list_indices_by_number[random_number])-1)
            group_i.append(list_indices_by_number[random_number][image_idx])
            y_label.append(0)
        X_groups.append(group_i)
        number_groups.append(numbers_i)
    
    #We now want our examples to have the following shape: (N, strip_size+1, X_train[1], X_train[2], 3)
    #And create the expected labels
    N = len(X_groups)
    img_size1 = X.shape[1]
    img_size2 = X.shape[2]
    X_processed= np.zeros([N, strip_size+1, img_size1, img_size2, 3])
    y_processed = np.zeros([N])
    for i in range(N):
        numbers_i = list(dict.fromkeys(number_groups[i]))
        for j in range(strip_size):
            X_processed[i, j:j+1, :, :, :] = X[X_groups[i][j]]
        X_processed[i, strip_size, :, :, :] = X[X_groups[i][strip_size]]
        y_processed[i] = y_label[i]
        
    return X_processed, y_processed

In [ ]:
def create_data_sets(X, y, Xt, yt, strip_size, RGB, training_size=20000, test_size=4000):
    if(RGB):
        Xtrain, ytrain = create_set_RGB(X, y, strip_size, set_size=training_size)
        Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, test_size=0.2)
        Xtest, ytest = create_set_RGB(Xt, yt, strip_size, set_size=test_size)
    else:
        #Transform the images to black and white
        Xtrain, ytrain = create_set(X, y, strip_size, set_size=training_size)
        Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, test_size=0.2)
        Xtest, ytest = create_set(Xt, yt, strip_size, set_size=test_size)
    
    """print ("Training examples classified as 0: " + str(len(np.where(ytrain==0)[0])))
    print ("Training examples classified as 1: " + str(len(np.where(ytrain==1)[0])))
    print ("Validation examples classified as 0: " + str(len(np.where(yval==0)[0])))
    print ("Validation examples classified as 1: " + str(len(np.where(yval==1)[0])))
    print ("Test examples classified as 0: " + str(len(np.where(ytest==0)[0])))
    print ("Test examples classified as 1: " + str(len(np.where(ytest==1)[0])))"""
    
    return Xtrain, Xval, Xtest, ytrain, yval, ytest

In [ ]:
def train_model(model, Xtrain, ytrain, Xval, yval, Xtest, ytest, lr=1e-3, batch_size=32, model_save_name="best_model"):
    
    #Define callbacks
    #Save the best model
    dirname = os.getcwd()
    filepath = os.path.join(dirname, model_save_name)
    filepath = os.path.join(filepath, 'model')
    model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss',
        mode='min', verbose = 0, save_best_only=True, save_weights_only=True)
    #Add early stopping
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, verbose = 0)
    #Reduce learning rate on plateau
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
    callbacks = [model_checkpoint_cb, early_stopping_cb, reduce_lr]
    
    #Compile and fit the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(Xtrain, ytrain,
                        batch_size=batch_size,
                        epochs=80,
                        validation_data=(Xval, yval),
                        callbacks=callbacks,
                        verbose=1)
    
    """plt.figure(figsize=(12,6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Val'], loc='upper right')


    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Val'], loc='upper right')"""
    
    model.load_weights(filepath)
    ypredict = model.predict(Xtest)
    #ypredict = tf.squeeze(ypredict).numpy()
    #print(ypredict)
    #ypredict_round = [round(x) for x in ypredict]
    score = model.evaluate(Xtest, ytest, verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])
    
    #cm = confusion_matrix(ytest, ypredict)
    #disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    #disp.plot()
    #plt.show()
    
    return score[1], model

In [ ]:
def save_to_db(db_name, column_name, list_to_save):
    df = pd.read_csv(db_name + '.csv')
    df[column_name] = list_to_save
    df.to_csv(db_name + '.csv', index=False)

def create_db(db_name):
    df = pd.DataFrame()
    df.to_csv(db_name + '.csv', index=True)

def plot_db_columns(db_name, title, xlabel, ylabel, save_name):
    df = pd.read_csv(db_name + '.csv')
    # plot lines
    x = [i for i in range(2, 11)]
    plt.figure(figsize=(9,7))
    
    for column in df:
        if (column != 'Unnamed: 0'):
            plt.plot(x, df[column], label = column)

    plt.title(title, fontsize=20)
    plt.xlabel(xlabel, fontsize=16)
    plt.ylabel(ylabel, fontsize=16)
    plt.legend()
    plt.grid(axis = 'y', color = 'gray', linestyle = '--', linewidth = 0.5)
    plt.tick_params(labelsize=14)
    plt.savefig(save_name + '.png')
    plt.show()

In [ ]:
def run_model(X, y, Xt, yt, model_class, latent_dim, model_save_name, db_name, column_name, preprocess_func, preprocess_before, num_iterations=3, RGB=True, pre_model=None, lr=1e-3):
    
    #Preprocess data
    if preprocess_before:
        X_processed = preprocess_func(X)
        Xt_processed = preprocess_func(Xt)
    else:
        X_processed = X
        Xt_processed = Xt
        
    accuracy_per_strip_size = []
    for strip_size in range(1, 10):
        print('-------------------' + str(strip_size) + '-------------------')
        channels = strip_size + 1
        accuracy_per_iteration = []
        for i in range(num_iterations):
            print('--------------Iteration ' + str(i+1) + '--------------')
            Xtrain, Xval, Xtest, ytrain, yval, ytest = create_data_sets(X_processed, y, Xt_processed, yt, strip_size, RGB=RGB)
            tf.keras.backend.clear_session()
            if pre_model:
                model = model_class(latent_dim, channels, (None, Xtrain[0].shape[1], Xtrain[0].shape[2], Xtrain[0].shape[3]), pre_model)
            else:
                model = model_class(latent_dim, channels, (None, Xtrain[0].shape[1], Xtrain[0].shape[2], Xtrain[0].shape[3])) 
            score, model = train_model(model, Xtrain, ytrain, Xval, yval, Xtest, ytest, lr=lr, batch_size=32, model_save_name=model_save_name)
            accuracy_per_iteration.append(score)
            del Xtrain
            del Xval
            del Xtest
            del ytrain
            del yval
            del ytest
            del model
        accuracy_per_strip_size.append(np.mean(accuracy_per_iteration))
    save_to_db(db_name, column_name, accuracy_per_strip_size)

# Creamos el csv donde guardaremos los datos

In [ ]:
create_db('cifar10_conv_encoder')

# Definimos los modelos

We will test the models with two different encoder blocks:
- If the model is named ---2, then it will be using the same encoder we used with the Fashion-MNIST dataset. 
- If the model is named ---3, then it will be using a new encoder better prepared for the Cifar10 dataset.

## Models with encoder 2
### With the dot product

In [ ]:
class ConvEncoderDot2(keras.Model):
    def __init__(self, latent_dim, channels, shape_in):
        super(ConvEncoderDot2, self).__init__()
        self.latent_dim = latent_dim
        self.channels = channels
        self.shape_in = shape_in
        
        self.encoder = tf.keras.Sequential([
            layers.Conv2D(32,(3,3), activation='relu', padding='same', input_shape=self.shape_in[1:]),
            layers.BatchNormalization(),
            layers.Conv2D(32, (3,3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPool2D(pool_size=(2,2)),
            
            layers.Conv2D(64, (3,3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Conv2D(64, (3,3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPool2D(pool_size=(2,2)),
            
            layers.Conv2D(128, (3,3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            
            layers.Conv2D(256, (3,3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            
            layers.Flatten(),
            
            layers.Dense(units=latent_dim)
            
        ])

    def call(self, x):
        
        encoded_images = layers.TimeDistributed(self.encoder)(x)
        
        if(self.channels > 2):
            max_image = layers.Maximum()([layers.Lambda(lambda x : x[:,i,:])(encoded_images) for i in range(self.channels-1)])
        else:
            max_image = layers.Lambda(lambda x : x[:,0,:])(encoded_images)
            
        last_embedding = layers.Lambda(lambda x : x[:,-1,:])(encoded_images)
        y_predict = layers.Dot(axes=1, normalize=True)([max_image, last_embedding])
        return y_predict

In [ ]:
class ConvEncoderDot2Pre(keras.Model):
    def __init__(self, latent_dim, channels, shape_in):
        super(ConvEncoderDot2Pre, self).__init__()
        self.latent_dim = latent_dim
        self.channels = channels
        self.shape_in = shape_in
        
        self.encoder = tf.keras.Sequential([
            layers.Conv2D(32,(3,3), activation='relu', padding='same', input_shape=[self.shape_in[1], self.shape_in[2], 1]),
            layers.BatchNormalization(),
            layers.Conv2D(32, (3,3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPool2D(pool_size=(2,2)),
            
            layers.Conv2D(64, (3,3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Conv2D(64, (3,3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPool2D(pool_size=(2,2)),
            
            layers.Conv2D(128, (3,3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            
            layers.Conv2D(256, (3,3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            
            layers.Flatten(),
            
            layers.Dense(units=latent_dim)
            
        ])
        
        self.pre_encoder = tf.keras.Sequential([
            layers.Conv2D(1, (1,1), padding='same', input_shape=self.shape_in[1:])
        ])

    def call(self, x):
        
        preencoded_images = layers.TimeDistributed(self.pre_encoder)(x)
        encoded_images = layers.TimeDistributed(self.encoder)(preencoded_images)
        
        if(self.channels > 2):
            max_image = layers.Maximum()([layers.Lambda(lambda x : x[:,i,:])(encoded_images) for i in range(self.channels-1)])
        else:
            max_image = layers.Lambda(lambda x : x[:,0,:])(encoded_images)
            
        last_embedding = layers.Lambda(lambda x : x[:,-1,:])(encoded_images)
        y_predict = layers.Dot(axes=1, normalize=True)([max_image, last_embedding])
        return y_predict

## Models with encoder 3

### With the dot product

In [ ]:
class ConvEncoderDot3(keras.Model):
    def __init__(self, latent_dim, channels, shape_in):
        super(ConvEncoderDot3, self).__init__()
        self.latent_dim = latent_dim
        self.channels = channels
        self.shape_in = shape_in
        
        self.encoder = keras.models.Sequential([
            layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001), input_shape=self.shape_in[1:]),
            layers.BatchNormalization(),
            layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.2),
            layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)),
            layers.BatchNormalization(),
            layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.3),
            layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)),
            layers.BatchNormalization(),
            layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.4),
            layers.Flatten(),
            layers.Dense(self.latent_dim, activation='relu', kernel_initializer='he_uniform'),
        ])

    def call(self, x):

        encoded_images = layers.TimeDistributed(self.encoder)(x)
        
        if(self.channels > 2):
            max_image = layers.Maximum()([layers.Lambda(lambda x : x[:,i,:])(encoded_images) for i in range(self.channels-1)])
        else:
            max_image = layers.Lambda(lambda x : x[:,0,:])(encoded_images)
            
        last_embedding = layers.Lambda(lambda x : x[:,-1,:])(encoded_images)
        y_predict = layers.Dot(axes=1, normalize=True)([max_image, last_embedding])
        return y_predict

In [ ]:
class ConvEncoderDot3Pre(keras.Model):
    def __init__(self, latent_dim, channels, shape_in):
        super(ConvEncoderDot3Pre, self).__init__()
        self.latent_dim = latent_dim
        self.channels = channels
        self.shape_in = shape_in
        
        self.encoder = keras.models.Sequential([
            layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001), input_shape=[self.shape_in[1], self.shape_in[2], 1]),
            layers.BatchNormalization(),
            layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.2),
            layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)),
            layers.BatchNormalization(),
            layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.3),
            layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)),
            layers.BatchNormalization(),
            layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.4),
            layers.Flatten(),
            layers.Dense(self.latent_dim, activation='relu', kernel_initializer='he_uniform'),
        ])

        self.pre_encoder = tf.keras.Sequential([
            layers.Conv2D(1, (1,1), padding='same', input_shape=self.shape_in[1:])
        ])

    def call(self, x):
        
        preencoded_images = layers.TimeDistributed(self.pre_encoder)(x)
        encoded_images = layers.TimeDistributed(self.encoder)(preencoded_images)
        
        if(self.channels > 2):
            max_image = layers.Maximum()([layers.Lambda(lambda x : x[:,i,:])(encoded_images) for i in range(self.channels-1)])
        else:
            max_image = layers.Lambda(lambda x : x[:,0,:])(encoded_images)
            
        last_embedding = layers.Lambda(lambda x : x[:,-1,:])(encoded_images)
        y_predict = layers.Dot(axes=1, normalize=True)([max_image, last_embedding])
        return y_predict

### With the dense classifier

In [ ]:
class ConvEncoderClassifier3(keras.Model):
    def __init__(self, latent_dim, channels, shape_in):
        super(ConvEncoderClassifier3, self).__init__()
        self.latent_dim = latent_dim
        self.channels = channels
        self.shape_in = shape_in
        
        self.encoder = keras.models.Sequential([
            layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001), input_shape=self.shape_in[1:]),
            layers.BatchNormalization(),
            layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.2),
            layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)),
            layers.BatchNormalization(),
            layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.3),
            layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)),
            layers.BatchNormalization(),
            layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.4),
            layers.Flatten(),
            layers.Dense(self.latent_dim, activation='relu', kernel_initializer='he_uniform'),
        ])
        
        self.classifier = tf.keras.Sequential([
            layers.Flatten(),
            #layers.Dense(512, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='sigmoid')
        ])

    def call(self, x):

        encoded_images = layers.TimeDistributed(self.encoder)(x)
        if self.channels > 2:
            max_image = layers.Maximum()([layers.Lambda(lambda x : x[:,i,:])(encoded_images) for i in range(self.channels-1)])
        else:
            max_image = layers.Lambda(lambda x : x[:,0,:])(encoded_images)

        target_image = layers.Lambda(lambda x : x[:,-1,:])(encoded_images)
        stacked_image = tf.stack([max_image, target_image], axis=-1)
        y_predict = self.classifier(stacked_image)
        return y_predict

In [ ]:
class ConvEncoderClassifier3Pre(keras.Model):
    def __init__(self, latent_dim, channels, shape_in):
        super(ConvEncoderClassifier3Pre, self).__init__()
        self.latent_dim = latent_dim
        self.channels = channels
        self.shape_in = shape_in
        
        self.encoder = keras.models.Sequential([
            layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001), input_shape=[self.shape_in[1], self.shape_in[2], 1]),
            layers.BatchNormalization(),
            layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.2),
            layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)),
            layers.BatchNormalization(),
            layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.3),
            layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)),
            layers.BatchNormalization(),
            layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.4),
            layers.Flatten(),
            layers.Dense(self.latent_dim, kernel_initializer='he_uniform'),
        ])
        
        self.classifier = tf.keras.Sequential([
            layers.Flatten(),
            #layers.Dense(512, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='sigmoid')
        ])

        self.pre_encoder = tf.keras.Sequential([
            layers.Conv2D(1, (1,1), padding='same', input_shape=self.shape_in[1:])
        ])

    def call(self, x):
        
        preencoded_images = layers.TimeDistributed(self.pre_encoder)(x)
        encoded_images = layers.TimeDistributed(self.encoder)(preencoded_images)
        
        if self.channels > 2:
            max_image = layers.Maximum()([layers.Lambda(lambda x : x[:,i,:])(encoded_images) for i in range(self.channels-1)])
        else:
            max_image = layers.Lambda(lambda x : x[:,0,:])(encoded_images)

        target_image = layers.Lambda(lambda x : x[:,-1,:])(encoded_images)
        stacked_image = tf.stack([max_image, target_image], axis=-1)
        y_predict = self.classifier(stacked_image)
        return y_predict

## Define the preprocessing functions

In [ ]:
def preprocess_flatten(images):
    flat_images = layers.Flatten()(images)
    reshape_images = layers.Reshape((images.shape[1], images.shape[2]*images.shape[3]))(flat_images)
    return reshape_images


def preprocess_maxchannels(images):
    max_images = layers.Maximum()([images[:, :, :, 0], images[:, :, :, 1], images[:, :, :, 2]])
    return max_images


def preprocess_identity(images):
    return images


def preprocess_black_and_white(images):
    b_w_images = tf.image.rgb_to_grayscale(images)
    return tf.squeeze(b_w_images)

# Train the models

In [ ]:
run_model(Xtrain_norm, ytrain_norm, Xtest_norm, ytest_norm, ConvEncoderDot2, 32, 'flatten_dot2', 'cifar10_conv_encoder', 'flatten_dot2', preprocess_flatten, True, num_iterations=1, RGB=False, pre_model=None, lr=1e-3)
run_model(Xtrain_norm, ytrain_norm, Xtest_norm, ytest_norm, ConvEncoderDot2, 32, 'max_dot2', 'cifar10_conv_encoder', 'max_dot2', preprocess_maxchannels, True, num_iterations=1, RGB=False, pre_model=None, lr=1e-3)
run_model(Xtrain_norm, ytrain_norm, Xtest_norm, ytest_norm, ConvEncoderDot2, 32, 'embedding_dot2', 'cifar10_conv_encoder', 'embedding_dot2', preprocess_identity, True, num_iterations=1, RGB=True, pre_model=None, lr=1e-3)
run_model(Xtrain_norm, ytrain_norm, Xtest_norm, ytest_norm, ConvEncoderDot2, 32, 'black_white_dot2', 'cifar10_conv_encoder', 'black_white_dot2', preprocess_black_and_white, True, num_iterations=1, RGB=False, pre_model=None, lr=1e-3)
run_model(Xtrain_norm, ytrain_norm, Xtest_norm, ytest_norm, ConvEncoderDot2Pre, 32, 'conv_dot2', 'cifar10_conv_encoder', 'conv_dot2', None, False, num_iterations=1, RGB=True, pre_model=None, lr=1e-3)


In [ ]:
#Modelos con encoder Cifar10 y dot product
run_model(Xtrain_norm, ytrain_norm, Xtest_norm, ytest_norm, ConvEncoderDot3, 32, 'embedding_dot3', 'cifar10_conv_encoder', 'embedding_dot3', preprocess_identity, True, num_iterations=1, RGB=True, pre_model=None, lr=1e-3)
run_model(Xtrain_norm, ytrain_norm, Xtest_norm, ytest_norm, ConvEncoderDot3, 32, 'black_white_dot3', 'cifar10_conv_encoder', 'black_white_dot3', preprocess_black_and_white, True, num_iterations=1, RGB=False, pre_model=None, lr=1e-3)
run_model(Xtrain_norm, ytrain_norm, Xtest_norm, ytest_norm, ConvEncoderDot3Pre, 32, 'conv_dot3', 'cifar10_conv_encoder', 'conv_dot3', None, False, num_iterations=1, RGB=True, pre_model=None, lr=1e-3)
run_model(Xtrain_norm, ytrain_norm, Xtest_norm, ytest_norm, ConvEncoderDot3, 32, 'flatten_dot3', 'cifar10_conv_encoder', 'flatten_dot3', preprocess_flatten, True, num_iterations=1, RGB=False, pre_model=None, lr=1e-3)
run_model(Xtrain_norm, ytrain_norm, Xtest_norm, ytest_norm, ConvEncoderDot3, 32, 'max_dot3', 'cifar10_conv_encoder', 'max_dot3', preprocess_maxchannels, True, num_iterations=1, RGB=False, pre_model=None, lr=1e-3)



In [ ]:
#Modelos con encoder Cifar10 y classifier
run_model(Xtrain_norm, ytrain_norm, Xtest_norm, ytest_norm, ConvEncoderClassifier3, 32, 'embedding_classifier3', 'cifar10_conv_encoder', 'embedding_classifier3', preprocess_identity, True, num_iterations=1, RGB=True, pre_model=None, lr=1e-3)
run_model(Xtrain_norm, ytrain_norm, Xtest_norm, ytest_norm, ConvEncoderClassifier3, 32, 'black_white_classifier3', 'cifar10_conv_encoder', 'black_white_classifier3', preprocess_black_and_white, True, num_iterations=1, RGB=False, pre_model=None, lr=1e-3)
run_model(Xtrain_norm, ytrain_norm, Xtest_norm, ytest_norm, ConvEncoderClassifier3Pre, 32, 'conv_classifier3', 'cifar10_conv_encoder', 'conv_classifier3', None, False, num_iterations=1, RGB=True, pre_model=None, lr=1e-3)
run_model(Xtrain_norm, ytrain_norm, Xtest_norm, ytest_norm, ConvEncoderClassifier3, 32, 'flatten_classifier3', 'cifar10_conv_encoder', 'flatten_classifier3', preprocess_flatten, True, num_iterations=1, RGB=False, pre_model=None, lr=1e-3)
run_model(Xtrain_norm, ytrain_norm, Xtest_norm, ytest_norm, ConvEncoderClassifier3, 32, 'max_classifier3', 'cifar10_conv_encoder', 'max_dot3', preprocess_maxchannels, True, num_iterations=1, RGB=False, pre_model=None, lr=1e-3)



# Plot results

In [ ]:
plot_db_columns('cifar10_conv_encoder', 'Cifar10 model accuracy', 'Strip size', 'Accuracy', 'cifar10_conv_encoder')